In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
import numpy as np
from tqdm import tqdm
import random
import os
from datasets import load_dataset

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Positional Encoding (unchanged but with explicit initialization for clarity)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super().__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Enhanced Transformer Model (modest changes)
class TransformerStoryModel(nn.Module):
    def __init__(self, vocab_size, d_model=384, nhead=8, num_encoder_layers=7, num_decoder_layers=7, dim_feedforward=1536, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size

        # Embeddings
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.embedding_dropout = nn.Dropout(dropout)

        # Encoder
        self.encoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'norm1': nn.LayerNorm(d_model),
                'attn': nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True),
                'norm2': nn.LayerNorm(d_model),
                'ffn': nn.Sequential(
                    nn.Linear(d_model, dim_feedforward),
                    nn.GELU(),
                    nn.Linear(dim_feedforward, d_model),
                    nn.Dropout(dropout)
                )
            }) for _ in range(num_encoder_layers)
        ])

        # Decoder
        self.decoder_layers = nn.ModuleList([
            nn.ModuleDict({
                'norm1': nn.LayerNorm(d_model),
                'self_attn': nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True),
                'norm2': nn.LayerNorm(d_model),
                'cross_attn': nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True),
                'norm3': nn.LayerNorm(d_model),
                'ffn': nn.Sequential(
                    nn.Linear(d_model, dim_feedforward),
                    nn.GELU(),
                    nn.Linear(dim_feedforward, d_model),
                    nn.Dropout(dropout)
                )
            }) for _ in range(num_decoder_layers)
        ])

        # Output layer
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.norm_out = nn.LayerNorm(d_model)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, mean=0, std=0.02)

    def create_mask(self, src, tgt):
        src_seq_len = src.shape[1]
        tgt_seq_len = tgt.shape[1]
        src_mask = torch.zeros((src_seq_len, src_seq_len), dtype=torch.bool, device=src.device)
        tgt_mask = self.generate_square_subsequent_mask(tgt_seq_len).to(tgt.device)
        src_padding_mask = (src == 0)
        tgt_padding_mask = (tgt == 0)
        return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, tgt):
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = self.create_mask(src, tgt)

        # Embeddings
        src_emb = self.token_embedding(src) * math.sqrt(self.d_model)
        tgt_emb = self.token_embedding(tgt) * math.sqrt(self.d_model)
        src_emb = self.positional_encoding(src_emb)
        tgt_emb = self.positional_encoding(tgt_emb)
        src_emb = self.embedding_dropout(src_emb)
        tgt_emb = self.embedding_dropout(tgt_emb)

        # Encoder
        memory = src_emb
        for layer in self.encoder_layers:
            memory = layer['norm1'](memory)
            memory = memory + layer['attn'](memory, memory, memory, attn_mask=src_mask, key_padding_mask=src_padding_mask)[0]
            memory = layer['norm2'](memory)
            memory = memory + layer['ffn'](memory)

        # Decoder
        output = tgt_emb
        for layer in self.decoder_layers:
            output = layer['norm1'](output)
            output = output + layer['self_attn'](output, output, output, attn_mask=tgt_mask, key_padding_mask=tgt_padding_mask)[0]
            output = layer['norm2'](output)
            output = output + layer['cross_attn'](output, memory, memory, key_padding_mask=src_padding_mask)[0]
            output = layer['norm3'](output)
            output = output + layer['ffn'](output)

        output = self.norm_out(output)
        return self.output_layer(output)

    def encode(self, src):
        src_emb = self.token_embedding(src) * math.sqrt(self.d_model)
        src_emb = self.positional_encoding(src_emb)
        src_emb = self.embedding_dropout(src_emb)
        src_padding_mask = (src == 0)
        memory = src_emb
        for layer in self.encoder_layers:
            memory = layer['norm1'](memory)
            memory = memory + layer['attn'](memory, memory, memory, key_padding_mask=src_padding_mask)[0]
            memory = layer['norm2'](memory)
            memory = memory + layer['ffn'](memory)
        return memory

    def decode(self, tgt, memory):
        tgt_emb = self.token_embedding(tgt) * math.sqrt(self.d_model)
        tgt_emb = self.positional_encoding(tgt_emb)
        tgt_emb = self.embedding_dropout(tgt_emb)
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        tgt_padding_mask = (tgt == 0)
        output = tgt_emb
        for layer in self.decoder_layers:
            output = layer['norm1'](output)
            output = output + layer['self_attn'](output, output, output, attn_mask=tgt_mask, key_padding_mask=tgt_padding_mask)[0]
            output = layer['norm2'](output)
            output = output + layer['cross_attn'](output, memory, memory)[0]
            output = layer['norm3'](output)
            output = output + layer['ffn'](output)
        output = self.norm_out(output)
        return self.output_layer(output)

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load TinyStories dataset
print("Loading TinyStories dataset...")
ds = load_dataset("roneneldan/TinyStories")

# Create tokenizer
tokenizer = SimpleTokenizer(tokenization_type='word')

# Fit tokenizer on a larger subset of the data
print("Fitting tokenizer on dataset...")
sample_size = 100000  # Use a larger sample for robust vocabulary
sample_texts = [ds["train"][i]["text"] for i in range(min(sample_size, len(ds["train"]))) if i < len(ds["train"])]
tokenizer.fit(sample_texts)

vocab_size = tokenizer.vocab_size()
print(f"Vocabulary size: {vocab_size}")

# Create datasets and dataloaders
seq_length = 128
# Create a single dataset from a larger subset of training data
max_samples = 100000  # Use 50,000 stories to ensure enough sequences
stories = [ds["train"][i]["text"] for i in range(min(max_samples, len(ds["train"])))]
dataset = StoryDataset(stories, tokenizer, seq_length=seq_length)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Calculate required batch size for ~1000 steps
target_steps = 1000
estimated_dataset_size = len(train_dataset)
batch_size = max(1, estimated_dataset_size // target_steps)  # Ensure at least 1
if batch_size > 32:  # Cap batch size to avoid memory issues
    batch_size = 32
print(f"Adjusted batch_size: {batch_size}, Estimated steps: {len(train_dataset) // batch_size}")

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, drop_last=False)

# Create model
model = TransformerStoryModel(
    vocab_size=vocab_size,
    d_model=256,      # Embedding dimension
    nhead=8,          # Number of attention heads
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=1024
).to(device)

# Setup optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * 10)

# Train model
trained_model = train_model(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    scheduler,
    device,
    epochs=5
)

# Save the final model
output_dir = "/kaggle/working/cusLLM_model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

final_model_path = os.path.join(output_dir, "final_model.pt")
torch.save({
    "model_state_dict": model.state_dict(),
    "tokenizer": {
        "word_to_idx": tokenizer.word_to_idx,
        "idx_to_word": tokenizer.idx_to_word,
        "tokenization_type": tokenizer.tokenization_type
    },
    "config": {
        "d_model": model.d_model,
        "vocab_size": model.vocab_size,
    }
}, final_model_path)
print(f"Final model saved to {final_model_path}")

Using device: cuda
Loading TinyStories dataset...


In [ ]:
# Define the correct path
final_model_path = "/kaggle/working/cusLLM_model/final_model.pt"

# Load checkpoint
checkpoint = torch.load(final_model_path, map_location=device)
print(list(checkpoint.keys()))
print(checkpoint["tokenizer"]["tokenization_type"])
# Rebuild the tokenizer from saved state
tokenizer = SimpleTokenizer(tokenization_type=checkpoint["tokenizer"]["tokenization_type"])
tokenizer.word_to_idx = checkpoint["tokenizer"]["word_to_idx"]
tokenizer.idx_to_word = checkpoint["tokenizer"]["idx_to_word"]

# Reinitialize the model with saved config
model = TransformerStoryModel(
    vocab_size=checkpoint["config"]["vocab_size"],
    d_model=checkpoint["config"]["d_model"],
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=1024
).to(device)

# Load model weights
model.load_state_dict(checkpoint["model_state_dict"])
print(f"Loaded model from {final_model_path}")

while True:
    # Get user input for text generation
    prompt = input("Enter a prompt for text generation: ")
    generated_text = generate_story(
        model,
        tokenizer,
        prompt,
        max_length=200,
        temperature=0.8,
        device=device
    )

    print(f"Generated text:\n{generated_text}")
    choice = input("next? (y/n):")
    if choice == "n":
        break

/tmp/ipykernel_31/1245307404.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(final_model_path, map_location=device)


['model_state_dict', 'tokenizer', 'config']
word
Loaded model from /kaggle/working/cusLLM_model/final_model.pt


Enter a prompt for text generation:  jim said


Generated text:
said and it of She little and a break. and bring of She forgot the and find came lived the and to had Lucy off my smiled give time star looked and a her excited the came named and smiled was a her little would and day, the but there and said and the a bird go her the and and lived play Her can hungry. and he and kept and flowers told their and he excited was of rolling the very to her a The play it happy The me the the was He a asked. tray other and and She soon. a the It's They a a on the It Lily made to to and took his she of showed it games the little was on loaded Tommy garden a then toy owl the said He He Anna was you, and a the that to the table. and Billy her needed the was it and not, friends. she and bird to the to was to He big tell saw and loved the bridge happy to and his be "You for very play finished, hot." The that was the and is a The for play. Ben a I'm the soon


next? (y/n): n
